In [ ]:
import os
from datetime import datetime
#from datetime import date
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
import pandas as pd
from numpy import int16


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query = '''
SELECT 
    dm.id_patient AS main_id,
    IF(TIMESTAMPDIFF(YEAR,
            dsd.nan_ki_dat_ou_fet,
            NOW()) >= 18,
        IF(dsd.a7_Ak_kiles_w_ap_viv_15_19 = '3'
                OR dsd.a1121_aktivite_pouw_rantre_kob_ou_vle_fe = '0'
                or dsd.c6b_Kiles_ki_peye_lekol_ou_Tranche_15_19 = '3'
                or dsd.eske_ou_bay_kob_pou_pran_swen_piti_ou_ayo = 'oui',
            'yes_sup18',
            'no18'),
        if(
			dsd.a7_ak_kils_w_ap_viv_10_14 = '6'
            OR dsd.a1121_aktivite_pouw_rantre_kob_ou_vle_fe = '0'
            OR dsd.c6b_kils_ki_peye_lekl_ou_10_14 = '5'
            or dsd.eske_ou_bay_kob_pou_pran_swen_piti_ou_ayo = 'oui',
            'yes_inf17'
            ,'no17'	)
		) AS muso_eligibility,
    dm.id_group AS actual_id_group,
    b.groups AS group_she_take_sessions,
    p.patient_code AS code,
    ben.last_name,
    ben.first_name,
    ben.dob,
    TIMESTAMPDIFF(YEAR, ben.dob, NOW()) AS age,
    b.pres AS number_of_different_topic,
    b.first_session_date,
    b.last_session_date,
    MAX(dhi.test_date) AS last_hiv_test_date,
    MAX(dhi.hiv_test_awareness_date) AS last_sensibilisation_hiv_test_date,
    GROUP_CONCAT(distinct dhi.hiv_test_acceptation, ',') acceptation_hiv_test,
    GROUP_CONCAT(DISTINCT dhi.test_result, ',') AS test_results,
    GROUP_CONCAT(DISTINCT ltlr.name, ',') AS test_results_with_label,
    MAX(dhi.condoms_reception_date) AS last_condoms_reception_date,
    group_concat(distinct dhi.has_been_sensibilize_for_condom, ',') as sensibilisation_condom,
    group_concat(distinct dhi.accept_condom, ',') as acceptation_condom,
    MAX(dhi.vbg_treatment_date) AS last_vbg_treatment_date,
    MAX(dhi.gynecological_care_date) AS last_gynecological_care_date,
    MAX(dhi.prep_awareness_date) AS last_sensibilisation_prep_date,
    MAX(dhi.prep_reference_date) AS last_reference_prep_date,
    MAX(dhi.prep_initiation_date) AS last_initiation_prep_date,
    GROUP_CONCAT(distinct dhi.prep_acceptation, ',') AS acceptation_prep,
    dg.name AS actual_group_name,
    dh.name AS actual_hub,
    lc.name AS actual_commune,
    dh.commune AS actual_commune_id,
    ld.name AS actual_departement,
    lc.departement AS actual_departement_id,
    IF(mgm.id_patient IS NOT NULL,
        'yes',
        'no') AS is_muso,
    IF(gb.case_id IS NOT NULL, 'yes', 'no') AS is_gardening,
    tf.*,
    dsd.*
FROM
    dream_member dm
        LEFT JOIN
    (SELECT 
        SUM(a.value = 'P') AS pres,
            a.id_patient,
            GROUP_CONCAT(DISTINCT a.id_group, ',') AS groups,
            MIN(a.date) AS first_session_date,
            MAX(a.date) AS last_session_date
    FROM
        (SELECT 
        dga.*, dgs.topic, dgs.date, dgs.id_group
    FROM
        dream_group_attendance dga
    LEFT JOIN dream_group_session dgs ON dgs.id = dga.id_group_session
    WHERE
        dga.value = 'P'
    GROUP BY dga.id_patient , dgs.topic) a
    GROUP BY a.id_patient) b ON b.id_patient = dm.id_patient
        LEFT JOIN
    beneficiary ben ON ben.id_patient = dm.id_patient
        LEFT JOIN
    patient p ON p.id = dm.id_patient
        LEFT JOIN
    caris_db.dream_hivinfos dhi ON dhi.id_patient = dm.id_patient
        LEFT JOIN
    lookup_testing_lab_result ltlr ON ltlr.id = dhi.test_result
        LEFT JOIN
    caris_db.dream_group dg ON dg.id = dm.id_group
        LEFT JOIN
    dream_hub dh ON dh.id = dg.id_dream_hub
        LEFT JOIN
    lookup_commune lc ON lc.id = dh.commune
        LEFT JOIN
    lookup_departement ld ON ld.id = lc.departement
        LEFT JOIN
    dreams_surveys_data dsd ON dsd.case_id = dm.case_id
        LEFT JOIN
    tracking_familymember tf ON tf.id_patient = dm.id_patient
        LEFT JOIN
    muso_group_members mgm ON mgm.id_patient = dm.id_patient
        LEFT JOIN
    gardening_beneficiary gb ON gb.code_dreams = p.patient_code
GROUP BY dm.id_patient
'''

# close the pool of connection
engine.dispose()

dreams_mastersheet = pd.read_sql_query(query,engine,parse_dates=True)
# get the test excel file from Query
dreams_mastersheet.to_excel('./dreams_services.xlsx',index=False,na_rep="NULL")

In [ ]:

# tranche d'age classique pr les services agyw
# tranche age mineur majeur pour les services agyw
# ovc age
def tranche_age_classique(age):
    """age classique"""
    if age>=10 and age<=14:
        return "10-14"
    elif age>=15 and age<=19:
        return "15-19"
    elif age>=20 and age<=24:
        return "20-24"
    elif age>=25 and age<=29:
        return "25-29"
    else:
        return "not_valid_age"
        
def tranche_age_mineur_majeur(age):
    """minor and major age"""
    if age>=10 and age<=17:
        return "10-17"
    elif age>=18 and age<=19:
        return "18-19"
    elif age>=20 and age<=24:
        return "20-24"
    elif age>=25 and age<=29:
        return "25-29"
    else:
        return "not_valid_age"
    
def ovc_age(age):
    """ovc valid age"""
    if age>=10 and age<=14:
        return "10-14"
    if age>=15 and age<=17:
        return "15-17"
    elif age>=18 and age<=24:
        return "not_ovc_age"
    else:
        return "not_valid_age"
    

def calculation_fiscalYear21(date):
    if date.year == 2021 and date.month>=1 and date.month<=3:
        return "FY21Q2"
    elif date.year == 2020 and date.month>=10 and date.month<=12:
        return "FY21Q1"
    elif date.year == 2021 and date.month>=4 and date.month<=6:
        return "FY21Q3"
    elif date.year == 2021 and date.month>=7 and date.month<=9:
        return "FY21Q4"
    elif date.year == 2020 and date.month>=4 and date.month<=6:
        return "FY20Q3"
    elif date.year == 2020 and date.month>=7 and date.month<=9:
        return "FY20Q4"
    else:
        return "not_valid"

def fiscalYear21(mod):
    if mod == "FY20Q3" or mod == "FY20Q4":
        return "Q3fy20-Q4fy20"
    else:
        return mod

    
def validTimeOnSystem(date):
    if date>= datetime.strptime("2020-04-01","%Y-%m-%d") and date<= datetime.now():
        return "required_Time_on"
    else:
        return "not_valid_time_on"
        
    

    
def between_now_date_entevyou(date):
    return (datetime.now().year - date.year) * 12 + (datetime.now().month - date.month)




def agywPeriods(months):
    if months <= 6:
        return "0-6 months"
    elif months>=7 and months<=12:
        return "7-12 months"
    elif months>=13 and months<=24:
        return "13-24 months"
    else:
        return "25+ months"
    

def curriculum_atLeastOneService(topics):
    return "servis_auMoins_1fois" if topics>=1 and topics<=19 else "zero_services_curriculum" #topics<=18


def status_curriculum(topics):
    if topics>=1 and topics<=13:
        return "curriculum incomplet"
    elif topics>=14 and topics<=19: #topics<=18
        return "curriculum complet"
    else:
        return "non-recu"

def id_quarter_services(date):
    if type(date) == type(pd.NaT):
        return 'errata'
    if (type(date) != type(pd.NaT)) and (date.year == 2021 and date.month>=1 and date.month<=3):
        return "FY21Q2"
    elif (type(date) != type(pd.NaT)) and (date.year == 2020 and date.month>=10 and date.month<=12):
        return "FY21Q1"
    elif (type(date) != type(pd.NaT)) and (date.year == 2021 and date.month>=4 and date.month<=6):
        return "FY21Q3"
    elif (type(date) != type(pd.NaT)) and (date.year == 2021 and date.month>=7 and date.month<=9):
        return "FY21Q4"
    elif (type(date) != type(pd.NaT)) and (date.year == 2020 and date.month>=1 and date.month<=3):
        return "FY20Q2"
    elif (type(date) != type(pd.NaT)) and (date.year == 2019 and date.month>=10 and date.month<=12):
        return "FY20Q1"
    elif (type(date) != type(pd.NaT)) and (date.year == 2020 and date.month>=4 and date.month<=6):
        return "FY20Q3"
    elif (type(date) != type(pd.NaT)) and (date.year == 2020 and date.month>=7 and date.month<=9):
        return "FY20Q4"
    elif (type(date) != type(pd.NaT)) and (date.year == 2019 and date.month>=1 and date.month<=3):
        return "FY19Q2"
    elif (type(date) != type(pd.NaT)) and (date.year == 2018 and date.month>=10 and date.month<=12):
        return "FY19Q1"
    elif (type(date) != type(pd.NaT)) and (date.year == 2019 and date.month>=4 and date.month<=6):
        return "FY19Q3"
    elif (type(date) != type(pd.NaT)) and (date.year == 2019 and date.month>=7 and date.month<=9):
        return "FY19Q4"
    else:
        return "not_valid_fy"
    
def valid_services(date):
    if type(date) == type(pd.NaT):
        return 'errata'
    elif (type(date) != type(pd.NaT))and(date.year==2020 or date.year==2021):
        return 'tested_on_given_date'
    else:
        return 'not_valid_date'
    
def condoms_services(df):
    return "sensOuRecu" if df.condoms=='tested_on_given_date' or df.sens_condom == "sens" else "no"
    
def acceptation_services(choices):
    if choices == "0,":
        return "refus"
    elif choices == '0,,1,' or choices == '1,':
        return "accept"
    else:
        return "not_valid"

def sensibilisation_services(choices):
    if choices == "0,":
        return "non-sens"
    elif choices == '0,,1,' or choices == '1,':
        return "sens"
    else:
        return "not_valid"
    
def post_care_app(df):
    return 'service_gyneco_vbg' if (df.vbg=="tested_on_given_date") or (df.gyneco=='tested_on_given_date') else 'no'

def socioEco_app(df):
    return 'service_muso_gardening' if (df.is_muso=='yes') or (df.is_gardening=='yes') else 'no'


def unServiceDreams(df):
     return '1_services_dreams_recus' if (df.curriculum_servis_auMoins_1fois == "servis_auMoins_1fois") or (df.condoms=='tested_on_given_date') or (df.hts=='tested_on_given_date') or (df.post_care_treatment=="service_gyneco_vbg") or (df.socio_eco_app=="service_muso_gardening") else 'no'

def new_unServiceDreams(df): 
     if (df.curriculum_servis_auMoins_1fois == "servis_auMoins_1fois") \
          or (df.new_condoms == "sensOuRecu") \
          or (df.hts=='tested_on_given_date') \
          or (df.post_care_treatment=="service_gyneco_vbg") \
          or (df.socio_eco_app=="service_muso_gardening") \
          or (df.init_prep=="tested_on_given_date"):
               return '1_services_dreams_recus'
     else: 
               return'no'
           

def service_primaire_10_14(df):
    return 'curriculum-servis' if (df.curriculum=="curriculum complet" and df.age_range == "10-14") else "no"

def service_primaire_15_19(df):
    return 'condoms&curriculum' if (df.curriculum=="curriculum complet" and df.age_range == "15-19" and df.condoms=='tested_on_given_date') else 'no'

def service_primaire_20_24(df):
    return 'condoms&hts&curriculum' if (df.curriculum=="curriculum complet" and df.condoms=='tested_on_given_date' and df.hts=='tested_on_given_date' and df.age_range == "20-24") else 'no'

def new_service_primaire_20_24(df):
    return 'condoms&curriculum' if (df.curriculum=="curriculum complet" and df.condoms=='tested_on_given_date' and df.age_range == "20-24") else 'no'
    
def new_service_primaire_15_19(df):
    return 'condoms&curriculum' if (df.curriculum=="curriculum complet" and df.new_condoms=="sensOuRecu" and df.age_range == "15-19") else 'no'


def newI_service_primaire_20_24(df):
    return 'condoms&curriculum' if (df.curriculum=="curriculum complet" and df.new_condoms=="sensOuRecu" and df.age_range == "20-24") else 'no'

def curriculum_condense(curriculum):
    return "curriculum_completed" if curriculum == "curriculum complet" else "curriculum_inc"

# AGE